In [9]:
import os
import re
import pandas as pd

base_folder = r"D:\PlantMaster\PlantMaster_Model\test_results"
all_dfs = []

def parse_classification_report(file_path):
    with open(file_path, "r", encoding="utf-8") as f:
        lines = f.readlines()

    data = []
    class_pattern = re.compile(r"^\s*(\S.*?)\s{2,}([\d\.]+)\s+([\d\.]+)\s+([\d\.]+)\s+(\d+)\s*$")

    for line in lines:
        match = class_pattern.match(line)
        if match:
            class_name = re.sub(r"_+", "_", match.group(1).strip())  # normalize underscores
            precision = float(match.group(2))
            recall = float(match.group(3))
            f1 = float(match.group(4))
            support = int(match.group(5))
            data.append([class_name, precision, recall, f1, support])

    return data


# Her model klasörü için
for model_folder in os.listdir(base_folder):
    model_path = os.path.join(base_folder, model_folder)
    if not os.path.isdir(model_path):
        continue

    for filename in os.listdir(model_path):
        if not filename.lower().startswith("test"):
            continue

        file_path = os.path.join(model_path, filename)
        data = parse_classification_report(file_path)
        tag = f"{model_folder}_{filename}"  # örn: resnet50_test_resnet50

        df = pd.DataFrame(data, columns=[
            "class",
            f"{tag}_precision",
            f"{tag}_recall",
            f"{tag}_f1",
            f"{tag}_support"
        ])
        all_dfs.append(df)

# Tüm DataFrame'leri 'class' üzerinden birleştir
merged_df = all_dfs[0]
for df in all_dfs[1:]:
    merged_df = pd.merge(merged_df, df, on="class", how="outer")

# CSV olarak kaydet
merged_df.to_csv("comparison_table.csv", index=False)
print("Karşılaştırmalı tablo oluşturuldu: comparison_table.csv")


Karşılaştırmalı tablo oluşturuldu: comparison_table.csv


In [13]:
merged_df.tail()

,class,dinov2_test_dinov2_precision,dinov2_test_dinov2_recall,dinov2_test_dinov2_f1,dinov2_test_dinov2_support,dinov2_test_manuel_dinov2_precision,dinov2_test_manuel_dinov2_recall,dinov2_test_manuel_dinov2_f1,dinov2_test_manuel_dinov2_support,efficientb4_test_efficientb4_precision,efficientb4_test_efficientb4_recall,efficientb4_test_efficientb4_f1,efficientb4_test_efficientb4_support,efficientb4_test_manuel_efficientb4_precision,efficientb4_test_manuel_efficientb4_recall,efficientb4_test_manuel_efficientb4_f1,efficientb4_test_manuel_efficientb4_support,mobilenet_test_manuel_mobilenet_precision,mobilenet_test_manuel_mobilenet_recall,mobilenet_test_manuel_mobilenet_f1,mobilenet_test_manuel_mobilenet_support,mobilenet_test_mobilenet_precision,mobilenet_test_mobilenet_recall,mobilenet_test_mobilenet_f1,mobilenet_test_mobilenet_support,resnet50_test_manuel_resnet50_precision,resnet50_test_manuel_resnet50_recall,resnet50_test_manuel_resnet50_f1,resnet50_test_manuel_resnet50_support,resnet50_test_resnet50_precision,resnet50_test_resnet50_recall,resnet50_test_resnet50_f1,resnet50_test_resnet50_support,vgg_test_manuel_vgg_precision,vgg_test_manuel_vgg_recall,vgg_test_manuel_vgg_f1,vgg_test_manuel_vgg_support,vgg_test_vgg_precision,vgg_test_vgg_recall,vgg_test_vgg_f1,vgg_test_vgg_support
35,Tomato_Tomato_Yellow_Leaf_Curl_Virus,0.9919,0.9959,0.9939,246,1.0000,0.7778,0.8750,9,1.0,0.99,1.0,246,0.33,0.11,0.17,9,0.33,0.11,0.17,9,1.0,1.0,1.0,246,0.00,0.00,0.00,9,1.0,1.0,1.0,246,0.00,0.00,0.00,9,1.0,1.0,1.0,246
36,Tomato_Tomato_mosaic_virus,0.9912,1.0000,0.9956,224,0.0000,0.0000,0.0000,5,1.0,1.00,1.0,224,0.00,0.00,0.00,5,0.00,0.00,0.00,5,1.0,1.0,1.0,224,0.00,0.00,0.00,5,1.0,1.0,1.0,224,0.00,0.00,0.00,5,1.0,1.0,1.0,224
37,Tomato_healthy,0.9877,0.9959,0.9917,241,0.5000,0.2000,0.2857,5,1.0,1.00,1.0,241,0.00,0.00,0.00,5,0.00,0.00,0.00,5,1.0,1.0,1.0,241,0.00,0.00,0.00,5,1.0,1.0,1.0,241,0.00,0.00,0.00,5,1.0,1.0,1.0,241
38,macro avg,0.9906,0.9905,0.9905,8802,0.5253,0.5531,0.4995,207,1.0,1.00,1.0,8802,0.35,0.36,0.29,207,0.35,0.35,0.28,207,1.0,1.0,1.0,8802,0.31,0.26,0.22,207,1.0,1.0,1.0,8802,0.26,0.27,0.21,207,1.0,1.0,1.0,8802
39,weighted avg,0.9908,0.9906,0.9906,8802,0.5166,0.5314,0.4916,207,1.0,1.00,1.0,8802,0.34,0.30,0.26,207,0.34,0.30,0.26,207,1.0,1.0,1.0,8802,0.29,0.21,0.19,207,1.0,1.0,1.0,8802,0.25,0.24,0.20,207,1.0,1.0,1.0,8802


In [22]:
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages

# Model isimlerini çıkar (örn: 'dinov2', 'resnet', ...)
model_names = set(col.split('_')[2] for col in merged_df.columns if col.startswith('dinov2_test_') or col.startswith('resnet50_test_') or col.startswith('efficientb4_test_') or col.startswith('vgg_test_') or col.startswith('mobilenet_test_'))

metrics = ['precision', 'recall', 'f1']

# PDF dosyasını aç
with PdfPages('model_comparison.pdf') as pdf:  # PDF dosyası adı
    # Her model için grafik
    for model in model_names:
        for metric in metrics:
            test_col = f'{model}_test_{model}_{metric}'
            manual_col = f'{model}_test_manuel_{model}_{metric}'

            if test_col in merged_df.columns and manual_col in merged_df.columns:
                # Son iki sınıfı çıkart
                classes = merged_df['class'].iloc[:-2]  # Son iki sınıfı çıkar
                test_scores = merged_df[test_col].iloc[:-2]  # Son iki değeri çıkar
                manual_scores = merged_df[manual_col].iloc[:-2]  # Son iki değeri çıkar

                x = range(len(classes))
                width = 0.35

                plt.figure(figsize=(max(12, len(classes) * 0.6), 6))  # Otomatik genişlik ayarı
                plt.bar([i - width/2 for i in x], test_scores, width=width, label='Veri Seti Test')
                plt.bar([i + width/2 for i in x], manual_scores, width=width, label='Dış Dünya Test')

                plt.xticks(x, classes, rotation=60, ha='right')
                plt.ylabel(metric.capitalize())
                plt.title(f'{model.upper()} - {metric.capitalize()} Karşılaştırması')
                plt.legend()
                plt.tight_layout()

                # Her grafiği PDF'ye kaydet
                pdf.savefig()  # Grafik PDF'ye kaydedilir
                plt.close()  # Grafiği kapat
